In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator

print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'faces-data-new')

print(dataset_path)

models_path = os.path.join(root_path, 'saved_models_new')
if not os.path.exists(models_path):
    os.mkdir(models_path)


tgt_pth = os.path.join(root_path, 'visualize_dcgan-v2')

if not os.path.exists(tgt_pth):
    os.mkdir(tgt_pth)



BATCH_SIZE = 32
img_rows, img_cols, img_depth = (196, 196, 3)


def rescaleFn(img):
    return img / 127.5 - 1


datagen=ImageDataGenerator(preprocessing_function=rescaleFn)


train_generator = datagen.flow_from_directory(
    directory=dataset_path,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    target_size=(img_rows, img_cols)
)


STEP_SIZE=train_generator.n//train_generator.batch_size


X = next(train_generator)
print(X.shape)

Using TensorFlow backend.


1.13.1
./faces-data-new
Found 3005 images belonging to 1 classes.
(32, 196, 196, 3)


In [2]:
img_shape = (196, 196, 3)
img_len = np.prod(img_shape)
latent_dim = img_len
noise_len = 100
input_dim = noise_len
ngf = 64
ndf = 64


def build_discriminator():
    return keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=(4, 4),
            strides=2,
            input_shape=img_shape,
            use_bias=False
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 2
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 4
        keras.layers.Conv2D(
            filters=ndf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 5
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(9, 9),
            strides=2,
            use_bias=False
        ),
        
        #output
        keras.layers.Activation(tf.nn.sigmoid),
        
        #flatten
        keras.layers.Flatten()
    ])


def build_generator():
    
    return keras.Sequential([
        # reshape 1d to 3d
        keras.layers.Reshape((10, 10, 1), input_shape=(input_dim,)),
        
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(4, 4),
            strides=1,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])


discriminator = build_discriminator()
generator = build_generator()

generator.summary()
discriminator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 10, 10, 1)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 22, 22, 512)       8192      
_________________________________________________________________
batch_normalization_4 (Batch (None, 22, 22, 512)       2048      
_________________________________________________________________
activation_6 (Activation)    (None, 22, 22, 512)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 46, 46, 256)       2097152   
_________________________________________________________________
batch_normalization_5 (Batch (None, 46, 46, 256)       1024      
_________________________________________________________________
acti

In [3]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator(x)
out = discriminator(z)

# GAN
gan = keras.models.Model(inputs=x, outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [4]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    0.5 * z_real[r_real * 4 + col] + 0.5
                )
            else:
                cell.imshow(
                    0.5 * z_fake[r_fake * 4 + col] + 0.5
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

In [ ]:
EPOCHS = 50001
STEPS = 1


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


# to be visualized
vis_noise = np.random.normal(size=(16, noise_len,))

# fake and real label
y_fake = np.zeros(shape=(BATCH_SIZE,))
y_real = np.ones(shape=(BATCH_SIZE,))


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        z = np.random.normal(size=(BATCH_SIZE, noise_len))
        x_fake = generator.predict(z)
        x_real = next(train_generator)
        
        if x_real.shape[0] < BATCH_SIZE:
            x_real = next(train_generator)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(x_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(x_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        z = np.random.normal(size=(BATCH_SIZE, noise_len))

        # train
        loss_1, acc_1 = gan.train_on_batch(z, y_real)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 25 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        visualizeGAN(e, x_real, viz_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, [D ACC: %100.00], [D LOSS: 0.04], [G ACC: %100.00], [G LOSS: 14.95]
E: 1, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %96.88], [G LOSS: 16.68]
E: 2, [D ACC: %100.00], [D LOSS: 0.06], [G ACC: %96.88], [G LOSS: 20.73]
E: 3, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %84.38], [G LOSS: 32.34]
E: 4, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %90.62], [G LOSS: 26.98]
E: 5, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %84.38], [G LOSS: 33.28]
E: 6, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %90.62], [G LOSS: 29.71]
E: 7, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %81.25], [G LOSS: 36.33]
E: 8, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %81.25], [G LOSS: 43.30]
E: 9, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %78.12], [G LOSS: 49.34]
E: 10, [D ACC: %100.00], [D LOSS: 0.23], [G ACC: %65.62], [G LOSS: 68.33]
E: 11, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %43.75], [G LOSS: 103.90]
E: 12, [D ACC: %100.00], [D LOSS: 0.21], [G ACC: %37.50], [G LOSS: 102.12]
E: 13, [D ACC: %100.00], [D LOSS: 0.33], [G A

E: 110, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %0.00], [G LOSS: 1523.83]
E: 111, [D ACC: %100.00], [D LOSS: 1.19], [G ACC: %0.00], [G LOSS: 1467.36]
E: 112, [D ACC: %98.44], [D LOSS: 5.25], [G ACC: %0.00], [G LOSS: 1320.85]
E: 113, [D ACC: %96.88], [D LOSS: 14.59], [G ACC: %3.12], [G LOSS: 1208.68]
E: 114, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %0.00], [G LOSS: 1028.70]
E: 115, [D ACC: %98.44], [D LOSS: 1.42], [G ACC: %9.38], [G LOSS: 988.13]
E: 116, [D ACC: %96.88], [D LOSS: 10.66], [G ACC: %9.38], [G LOSS: 954.22]
E: 117, [D ACC: %96.88], [D LOSS: 4.70], [G ACC: %9.38], [G LOSS: 805.33]
E: 118, [D ACC: %96.88], [D LOSS: 12.03], [G ACC: %34.38], [G LOSS: 422.62]
E: 119, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %46.88], [G LOSS: 233.12]
E: 120, [D ACC: %100.00], [D LOSS: 0.58], [G ACC: %75.00], [G LOSS: 72.65]
E: 121, [D ACC: %100.00], [D LOSS: 0.60], [G ACC: %100.00], [G LOSS: 1.74]
E: 122, [D ACC: %98.44], [D LOSS: 2.66], [G ACC: %96.88], [G LOSS: 12.85]
E: 123, [D ACC: %96.88

E: 220, [D ACC: %100.00], [D LOSS: 0.04], [G ACC: %100.00], [G LOSS: 0.18]
E: 221, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.54]
E: 222, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.20]
E: 223, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %100.00], [G LOSS: 0.35]
E: 224, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %100.00], [G LOSS: 0.23]
E: 225, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.79]
E: 226, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.93]
E: 227, [D ACC: %100.00], [D LOSS: 0.06], [G ACC: %100.00], [G LOSS: 0.37]
E: 228, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %100.00], [G LOSS: 0.18]
E: 229, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.76]
E: 230, [D ACC: %100.00], [D LOSS: 0.04], [G ACC: %100.00], [G LOSS: 0.63]
E: 231, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 0.76]
E: 232, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 1.58]
E: 233, [D ACC: %100.00],

E: 329, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 330, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 331, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %0.00], [G LOSS: 1611.81]
E: 332, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 333, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %0.00], [G LOSS: 1611.81]
E: 334, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 335, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 336, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 337, [D ACC: %100.00], [D LOSS: 0.61], [G ACC: %0.00], [G LOSS: 1611.81]
E: 338, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 339, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 340, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 341, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 342, [D A

...

E: 27985, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27986, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27987, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27988, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27989, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27990, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27991, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27992, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27993, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27994, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27995, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27996, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 27997, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G L

E: 28091, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28092, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28093, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28094, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28095, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28096, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28097, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28098, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28099, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28100, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28101, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28102, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28103, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G L

E: 28197, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28198, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28199, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28200, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28201, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28202, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28203, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28204, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28205, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28206, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28207, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28208, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28209, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G L

E: 28303, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28304, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28305, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28306, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28307, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28308, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28309, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28310, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28311, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28312, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28313, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28314, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28315, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G L

E: 28409, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28410, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28411, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28412, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28413, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28414, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28415, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28416, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28417, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28418, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28419, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28420, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1611.81]
E: 28421, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G L

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();